In [5]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
from itertools import product

In [6]:
def sort_df(length, df):
    df = df.drop_duplicates().reset_index(drop=True)

    new_columns = [f'Col_{i}' for i in range(1, length + 1)]
    df_sort = df.sort_values(by = new_columns).reset_index(drop=True)
    for i in df_sort.columns:
        df_sort[i] = df_sort[i].astype(int)
        
    return df_sort

In [7]:
def build_data_to_df(size,amount_boards,amount_moves,num_dict,ignore_range,path,gen,reverse):
    os.makedirs(f'{path}\\{size}-{amount_boards}',exist_ok=True)
    df2 = split_board_to_series_df(size,amount_boards,amount_moves,num_dict,gen,ignore_range,reverse)
    df2_sort = sort_df(gen*size*size, df2)
    if reverse==True:
        name_df = f'{path}\\{size}-{amount_boards}\\{size}size_{amount_boards}boards_{gen}gen_reverse'
    else:
        name_df = f'{path}\\{size}-{amount_boards}\\{size}size_{amount_boards}boards_{gen}gen_non-reverse'
    df2_sort.to_pickle(f'{name_df}.pkl')
    print("\nfinish build df ",gen, reverse)

In [ ]:
SIZE = 5
AMOUNT_BOARDS = 10000
AMOUNT_MOVES = 100
NUM_DICT = 1
for gen,reverse in product(range(2,5),[True,False]):
    print(gen,reverse)
    build_data_to_df(SIZE, AMOUNT_BOARDS, AMOUNT_MOVES, NUM_DICT, IGNORE_RANGE, PATH_DF, gen, reverse)

2 True
0 
50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450 1500 1550 1600 1650 1700 1750 1800 1850 1900 1950 2000 2050 2100 2150 2200 2250 2300 2350 2400 2450 2500 2550 2600 2650 2700 2750 2800 2850 2900 2950 3000 3050 3100 3150 3200 3250 3300 3350 3400 3450 3500 3550 3600 3650 3700 3750 3800 3850 3900 3950 4000 4050 4100 4150 4200 4250 4300 4350 4400 4450 4500 4550 4600 4650 4700 4750 4800 4850 4900 4950 5000 5050 5100 5150 5200 5250 5300 5350 5400 5450 5500 5550 5600 5650 5700 5750 5800 5850 5900 5950 6000 6050 6100 6150 6200 6250 6300 6350 6400 6450 6500 6550 6600 6650 6700 6750 6800 6850 6900 6950 7000 7050 7100 7150 7200 7250 7300 7350 7400 7450 7500 7550 7600 7650 7700 7750 7800 7850 7900 7950 8000 8050 8100 8150 8200 8250 8300 8350 8400 8450 8500 8550 8600 8650 8700 8750 8800 8850 8900 8950 9000 9050 9100 9150 9200 9250 9300 9350 9400 9450 9500 9550 9600 9650 9700 9750 9800 9850 9900 9950 
finish build d